In [111]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
%matplotlib inline


In [19]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True)
       )

In [20]:
load_dotenv()

PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')


# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [21]:
INSTITUTION_ID = "ins_109508"


In [28]:
# YOUR CODE HERE

PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_SBX_SECRET_KEY = os.getenv('PLAID_SECRET')

client = plaid.Client(
    client_id=PLAID_CLIENT_ID, 
    secret=PLAID_SBX_SECRET_KEY, 
    public_key=PLAID_PUBLIC_KEY, 
    environment='sandbox')



### 2. Generate a public token

In [30]:
# YOUR CODE HERE

create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])
create_tkn_response


{'public_token': 'public-sandbox-3ce01dba-41ca-41a1-a02e-5dc75794000c',
 'request_id': 'rqqC0rXQbLAhqiy'}

### 3. Exchange the public token for an access token

In [31]:
# YOUR CODE HERE

exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])
exchange_response


{'access_token': 'access-sandbox-ce580750-9991-4a41-83a2-3fc25563ed2a',
 'item_id': 'NXE4JwXm44Hpem18abwpfradbZ144LfW6aaZy',
 'request_id': 'yG13C1lssEQCgty'}

### 4. Fetch Accounts

In [32]:
# YOUR CODE HERE

access_token = exchange_response['access_token']

client.Accounts.get(access_token)



{'accounts': [{'account_id': 'KXoxJaXGxxHMaGBALVpMcRK7kvDPrltVokbDX',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'rPxBvRPABBSaQMxAoVGaC4m1zLE7wWClb9gke',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'zLejvMLRjjHZ84WmdQMZsvBrjNAgVZcodBWXG',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [156]:
# YOUR CODE HERE

start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

transaction_response = client.Transactions.get(access_token,start_date,end_date)


print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))



[
    {
        "account_id": "KXoxJaXGxxHMaGBALVpMcRK7kvDPrltVokbDX",
        "account_owner": null,
        "amount": 89.4,
        "authorized_date": null,
        "category": [
            "Food and Drink",
            "Restaurants"
        ],
        "category_id": "13005000",
        "date": "2020-04-11",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "SparkFun",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": false,
     

### 2. Print the categories for each transaction

In [152]:
# YOUR CODE HERE

breakdown = json.dumps(category_response[category],indent=4, sort_keys=True)
print(breakdown)



[
    {
        "account_id": "KXoxJaXGxxHMaGBALVpMcRK7kvDPrltVokbDX",
        "balances": {
            "available": 100,
            "current": 110,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "0000",
        "name": "Plaid Checking",
        "official_name": "Plaid Gold Standard 0% Interest Checking",
        "subtype": "checking",
        "type": "depository"
    },
    {
        "account_id": "rPxBvRPABBSaQMxAoVGaC4m1zLE7wWClb9gke",
        "balances": {
            "available": 200,
            "current": 210,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "1111",
        "name": "Plaid Saving",
        "official_name": "Plaid Silver Standard 0.1% Interest Saving",
        "subtype": "savings",
        "type": "depository"
    },
    {
        "account_id": "zLejvMLRjjHZ84WmdQMZsvBrjNAgVZcodBWXG"

In [145]:
extracted = []
for row in client_past_90_day_transactions[1]:
    extracted.append([row["category"],row["amount"]])

TypeError: string indices must be integers

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [153]:
# YOUR CODE HERE

selected_value = client.transactions['category']
print(selected_value)

AttributeError: 'Client' object has no attribute 'transactions'

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [ ]:
# YOUR CODE HERE



---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# YOUR CODE HERE




---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE



### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE

